# IMPORT

In [ ]:
%pip install xgboost lightgbm catboost optuna

In [ ]:
# ADVANCED TREE-BASED ENSEMBLE WITH STACKING
# Combines LightGBM, XGBoost, and CatBoost with meta-learning

import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from sklearn.model_selection import KFold, GroupKFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error
import optuna  # For hyperparameter optimization
warnings.filterwarnings('ignore')

print("="*60)
print("ADVANCED TREE-BASED STACKING PIPELINE")
print("="*60)

# ============================================
# 1. LOAD AND PREPARE DATA
# ============================================

print("\n📁 Loading data...")

train_df = pd.read_csv('/content/train_data.csv')
test_df_original = pd.read_csv('/content/test_data_masked.csv')
systems_df = pd.read_csv('/content/systems_new.csv')
sample_submission = pd.read_csv('/content/sample_submissions.csv')

# Convert timestamps
train_df['timestamp'] = pd.to_datetime(train_df['timestamp'])
test_df_original['timestamp'] = pd.to_datetime(test_df_original['timestamp'])

# Fix capacity units
systems_df['panels_capacity_W'] = systems_df['panels_capacity'] * 1000
systems_df['load_capacity_W'] = systems_df['load_capacity'] * 1000

print(f"✅ Loaded {len(train_df):,} training samples")

# EDA

In [ ]:
print("Train date range:", train_df["timestamp"].min(), "to", train_df["timestamp"].max())
print("Test date range:", test_df_original["timestamp"].min(), "to", test_df_original["timestamp"].max())
print("Train systems:", train_df["system_id"].nunique())
print("Test systems:", test_df_original["system_id"].nunique())
print("Metadata systems:", systems_df["system_id"].nunique())

# Check alignment
print("Systems missing in train:", set(systems_df["system_id"]) - set(train_df["system_id"]))
print("Systems missing in test:", set(systems_df["system_id"]) - set(test_df_original["system_id"]))

In [ ]:
# NaNs
print(train_df.isna().sum())

# Placeholders
print("Placeholder counts (train):")
print((train_df[["generation_W","load_W"]] < 0).sum())

print("Placeholder counts (test):")
print((test_df_original[["generation_W","load_W"]] < 0).sum())

# Per system missing
missing_summary = train_df.groupby("system_id")[["generation_W","load_W"]].apply(lambda x: (x<=0).sum())
print(missing_summary.head())

In [ ]:
def check_time_gaps(df, sys_id):
    sub = df[df["system_id"] == sys_id].sort_values("timestamp").copy() # Add .copy() to avoid SettingWithCopyWarning
    sub["timestamp"] = pd.to_datetime(sub["timestamp"]) # Convert to datetime objects
    diffs = sub["timestamp"].diff().dropna()
    return diffs.value_counts()

# Convert timestamp columns in train and test dataframes to datetime objects
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])
test_df_original["timestamp"] = pd.to_datetime(test_df_original["timestamp"])

for sys in train_df["system_id"].unique()[:3]:  # sample a few
    print("System", sys, check_time_gaps(train_df, sys).head())

# Any duplicates?
dup = train_df.duplicated(subset=["system_id","timestamp"]).sum()
print("Duplicate rows:", dup)

In [ ]:
# Merge capacity metadata
train_df = train_df.merge(systems_df, on="system_id", how="left")

# Negative values
negatives = (train_df[["generation_W","load_W"]] < 0).sum()
print("Negative counts:\n", negatives)

# Outliers vs capacity
train_df["gen_capacity_ratio"] = train_df["generation_W"] / (train_df["panels_capacity"]*1000)
train_df["load_capacity_ratio"] = train_df["load_W"] / (train_df["load_capacity"]*1000)

print(train_df["gen_capacity_ratio"].describe())
print(train_df["load_capacity_ratio"].describe())

# Flag suspicious
suspicious = train_df[(train_df["gen_capacity_ratio"] > 1.5) | (train_df["gen_capacity_ratio"] < 0)]
print("Suspicious rows:", suspicious.shape)

In [ ]:
train_df["hour"] = train_df["timestamp"].dt.hour
train_df["month"] = train_df["timestamp"].dt.month

hourly = train_df.groupby("hour")[["generation_W","load_W"]].mean()
monthly = train_df.groupby("month")[["generation_W","load_W"]].mean()

hourly.plot(title="Avg Generation/Load by Hour", figsize=(10,5))
monthly.plot(title="Avg Generation/Load by Month", figsize=(10,5))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt # Import matplotlib for plotting

sns.boxplot(data=train_df, x="connection_type", y="generation_W")
plt.title("Generation Distribution by Connection Type")
plt.show()

sns.boxplot(data=train_df, x="location", y="generation_W")
plt.title("Generation Distribution by City")
plt.xticks(rotation=45)
plt.show()

# FEATURE ENGINEERING AND SELECTION

In [ ]:
# ============================================
# 2. ENHANCED FEATURE ENGINEERING
# ============================================

def create_advanced_features(df, systems_df, top_locations=None):
    """Create more sophisticated features with consistent location encoding"""

    # Merge with systems
    df = df.merge(systems_df[['system_id', 'panels_capacity_W', 'load_capacity_W',
                              'connection_type', 'location']],
                  on='system_id', how='left')

    # Time features
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    df['day_of_year'] = df['timestamp'].dt.dayofyear
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    df['month'] = df['timestamp'].dt.month
    df['quarter'] = df['timestamp'].dt.quarter
    df['week_of_year'] = df['timestamp'].dt.isocalendar().week

    # Cyclical encoding
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # Enhanced solar features
    df['solar_elevation'] = np.where(
        (df['hour'] >= 6) & (df['hour'] <= 18),
        np.sin(np.pi * (df['hour'] - 6) / 12),
        0
    )

    # Solar intensity approximation (considers time and season)
    summer_factor = np.sin(np.pi * (df['day_of_year'] - 80) / 365)  # Peak around June
    df['solar_intensity'] = df['solar_elevation'] * (1 + 0.3 * summer_factor)
    df['solar_intensity'] = df['solar_intensity'].clip(0, 1.3)

    # Peak hours with granularity
    df['is_morning_peak'] = ((df['hour'] >= 7) & (df['hour'] <= 9)).astype(int)
    df['is_solar_peak'] = ((df['hour'] >= 11) & (df['hour'] <= 14)).astype(int)
    df['is_afternoon_peak'] = ((df['hour'] >= 15) & (df['hour'] <= 17)).astype(int)
    df['is_evening_peak'] = ((df['hour'] >= 18) & (df['hour'] <= 21)).astype(int)
    df['is_night'] = ((df['hour'] <= 5) | (df['hour'] >= 22)).astype(int)

    # System features
    df['is_residential'] = (df['connection_type'] == 'RESIDENTIAL').astype(int)
    df['is_commercial'] = (df['connection_type'] == 'COMMERCIAL').astype(int)

    # Capacity features
    df['capacity_ratio'] = df['panels_capacity_W'] / (df['load_capacity_W'] + 1)
    df['log_panels_capacity'] = np.log1p(df['panels_capacity_W'])
    df['log_load_capacity'] = np.log1p(df['load_capacity_W'])
    df['total_capacity'] = df['panels_capacity_W'] + df['load_capacity_W']

    # Interaction features
    df['solar_x_capacity'] = df['solar_elevation'] * df['panels_capacity_W']
    df['hour_x_residential'] = df['hour'] * df['is_residential']
    df['hour_x_commercial'] = df['hour'] * df['is_commercial']
    df['weekend_x_residential'] = df['day_of_week'].isin([5, 6]).astype(int) * df['is_residential']

    # FIX: Location encoding with consistent features
    if top_locations is not None:
        # Use provided top locations (from training data)
        for loc in top_locations:
            df[f'loc_{loc}'] = (df['location'] == loc).astype(int)
    else:
        # This is training data, determine top locations
        top_locations = df['location'].value_counts().head(10).index
        for loc in top_locations:
            df[f'loc_{loc}'] = (df['location'] == loc).astype(int)

    # Advanced time features
    df['minutes_from_noon'] = np.abs((df['hour'] * 60 + df['minute']) - 720)
    df['minutes_from_midnight'] = np.minimum(
        df['hour'] * 60 + df['minute'],
        1440 - (df['hour'] * 60 + df['minute'])
    )

    return df, top_locations

# REPLACE the feature engineering section with this:

print("\n🔧 Creating advanced features...")

# Process training data and get top locations
train_processed, top_locations_from_train = create_advanced_features(train_df.copy(), systems_df.copy())

# Process test data using the same top locations from training
test_processed, _ = create_advanced_features(test_df_original.copy(), systems_df.copy(), top_locations=top_locations_from_train)

# CRITICAL: Normalize targets - Moved before feature selection
train_processed['generation_normalized'] = (
    train_processed['generation_W'] / (train_processed['panels_capacity_W'] + 1)
).clip(0, 1.5)
train_processed['load_normalized'] = (
    train_processed['load_W'] / (train_processed['load_capacity_W'] + 1)
).clip(0, 2.0)

print(f"✅ Created {len(train_processed.columns)} features")
print(f"✅ Top locations used: {list(top_locations_from_train)}")

# Now verify features match - Adjusted logic
train_cols = set(train_processed.columns)
test_cols = set(test_processed.columns)

# Identify columns unique to each set (excluding test_id from this check)
unique_to_train = train_cols - test_cols - {'generation_normalized', 'load_normalized'}
unique_to_test = test_cols - train_cols - {'test_id'}

if unique_to_train:
    print(f"⚠️ Columns unique to train: {unique_to_train}")
    # Drop columns unique to train from train_processed (except target variables)
    train_processed = train_processed.drop(columns=list(unique_to_train), errors='ignore')

if unique_to_test:
    print(f"⚠️ Columns unique to test: {unique_to_test}")
    # Drop columns unique to test from test_processed
    test_processed = test_processed.drop(columns=list(unique_to_test), errors='ignore')

# Ensure column order is the same for train and test (excluding test_id from test)
common_cols = [col for col in train_processed.columns if col in test_processed.columns]
train_processed = train_processed[common_cols + ['generation_normalized', 'load_normalized']] # Keep targets in train
test_processed = test_processed[common_cols + ['test_id']] # Keep test_id in test

# Add a check to ensure feature columns match after dropping
feature_cols_train_check = set(train_processed.columns) - {'generation_normalized', 'load_normalized'}
feature_cols_test_check = set(test_processed.columns) - {'test_id'}

if feature_cols_train_check != feature_cols_test_check:
     print("❌ Error: Feature columns in train_processed and test_processed do not match after dropping!")
     print("Features in train_processed:", sorted(list(feature_cols_train_check)))
     print("Features in test_processed:", sorted(list(feature_cols_test_check)))
else:
     print("✅ Feature columns in train_processed and test_processed match.")


# ============================================
# 3. FEATURE SELECTION
# ============================================

# Select features (exclude identifiers and targets)
exclude_cols = ['timestamp', 'system_id', 'generation_W', 'load_W',
                'connection_type',
                'location', 'test_id', 'generation_normalized', 'load_normalized'] # Explicitly exclude normalized targets
feature_cols = [col for col in train_processed.columns if col not in exclude_cols]

print(f"📊 Using {len(feature_cols)} features for modeling")

X_train = train_processed[feature_cols]
y_train_gen = train_processed['generation_normalized']
y_train_load = train_processed['load_normalized']
X_test = test_processed[feature_cols]

# MODEL TRAINING

In [ ]:
# ============================================
# 4. LIGHTGBM WITH HYPERPARAMETER TUNING
# ============================================

print("\n" + "="*60)
print("OPTIMIZED LIGHTGBM MODELS")
print("="*60)

import lightgbm as lgb

def train_lgb_with_cv(X, y, params=None, n_folds=5):
    """Train LightGBM with cross-validation"""

    if params is None:
        params = {
            'objective': 'regression',
            'metric': 'mae',
            'boosting_type': 'gbdt',
            'num_leaves': 50,
            'learning_rate': 0.02,
            'feature_fraction': 0.7,
            'bagging_fraction': 0.7,
            'bagging_freq': 5,
            'min_child_samples': 20,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'verbose': -1,
            'num_threads': -1
        }

    # Cross-validation for robust training
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    models = []
    scores = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
        X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
        y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]

        train_data = lgb.Dataset(X_fold_train, label=y_fold_train)
        val_data = lgb.Dataset(X_fold_val, label=y_fold_val, reference=train_data)

        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=1000,
            callbacks=[
                lgb.early_stopping(50),
                lgb.log_evaluation(0)
            ]
        )

        models.append(model)
        val_pred = model.predict(X_fold_val, num_iteration=model.best_iteration)
        mae = mean_absolute_error(y_fold_val, val_pred)
        scores.append(mae)
        print(f"  Fold {fold} MAE: {mae:.5f}")

    print(f"  Average MAE: {np.mean(scores):.5f} (±{np.std(scores):.5f})")
    return models

# Train generation models
print("\n📊 Training LightGBM for generation...")
lgb_gen_models = train_lgb_with_cv(X_train, y_train_gen)

# Train load models
print("\n📊 Training LightGBM for load...")
lgb_load_models = train_lgb_with_cv(X_train, y_train_load)

# ============================================
# 5. XGBOOST MODELS
# ============================================

print("\n" + "="*60)
print("XGBOOST MODELS")
print("="*60)

try:
    import xgboost as xgb

    def train_xgb_with_cv(X, y, n_folds=5):
        """Train XGBoost with cross-validation"""

        params = {
            'objective': 'reg:absoluteerror',
            'max_depth': 6,
            'learning_rate': 0.02,
            'n_estimators': 1000,
            'subsample': 0.7,
            'colsample_bytree': 0.7,
            'reg_alpha': 0.1,
            'reg_lambda': 0.1,
            'random_state': 42,
            'n_jobs': -1,
            'verbosity': 0
        }

        kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
        models = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
            X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
            y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]

            model = xgb.XGBRegressor(
                **params,
                eval_metric='mae',
                early_stopping_rounds=50
            )
            model.fit(
                X_fold_train, y_fold_train,
                eval_set=[(X_fold_val, y_fold_val)],
                verbose=False
            )

            models.append(model)
            print(f"  Fold {fold} trained")

        return models

    print("📊 Training XGBoost for generation...")
    xgb_gen_models = train_xgb_with_cv(X_train, y_train_gen)

    print("📊 Training XGBoost for load...")
    xgb_load_models = train_xgb_with_cv(X_train, y_train_load)

    USE_XGBOOST = True
except ImportError:
    print("⚠️ XGBoost not available")
    USE_XGBOOST = False

# ============================================
# 6. CATBOOST MODELS
# ============================================

print("\n" + "="*60)
print("CATBOOST MODELS")
print("="*60)

try:
    import catboost as cb

    def train_catboost_with_cv(X, y, n_folds=3):
        """Train CatBoost with cross-validation"""

        params = {
            'loss_function': 'MAE',
            'iterations': 1000,
            'learning_rate': 0.03,
            'depth': 6,
            'l2_leaf_reg': 3,
            'random_seed': 42,
            'verbose': False
        }

        kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
        models = []

        for fold, (train_idx, val_idx) in enumerate(kf.split(X), 1):
            X_fold_train, X_fold_val = X.iloc[train_idx], X.iloc[val_idx]
            y_fold_train, y_fold_val = y.iloc[train_idx], y.iloc[val_idx]

            model = cb.CatBoostRegressor(**params)
            model.fit(
                X_fold_train, y_fold_train,
                eval_set=(X_fold_val, y_fold_val),
                early_stopping_rounds=50,
                verbose=False
            )

            models.append(model)
            print(f"  Fold {fold} trained")

        return models

    print("📊 Training CatBoost for generation...")
    cb_gen_models = train_catboost_with_cv(X_train, y_train_gen)

    print("📊 Training CatBoost for load...")
    cb_load_models = train_catboost_with_cv(X_train, y_train_load)

    USE_CATBOOST = True
except ImportError:
    print("⚠️ CatBoost not available")
    USE_CATBOOST = False

# ============================================
# 7. STACKING WITH META-LEARNER
# ============================================

print("\n" + "="*60)
print("STACKING ENSEMBLE")
print("="*60)

def create_meta_features(models_dict, X, model_type='lgb'):
    """Create predictions from base models for meta-learner"""
    predictions = []

    for models in models_dict[model_type]:
        if model_type == 'lgb':
            pred = models.predict(X, num_iteration=models.best_iteration)
        else:
            pred = models.predict(X)
        predictions.append(pred)

    # Return mean and std of predictions
    preds_array = np.array(predictions)
    return np.mean(preds_array, axis=0), np.std(preds_array, axis=0)

# Create meta features for training
print("Creating meta features...")
meta_train_gen = []
meta_train_load = []

# LightGBM predictions
lgb_gen_mean, lgb_gen_std = create_meta_features({'lgb': lgb_gen_models}, X_train, 'lgb')
lgb_load_mean, lgb_load_std = create_meta_features({'lgb': lgb_load_models}, X_train, 'lgb')
meta_train_gen.extend([lgb_gen_mean, lgb_gen_std])
meta_train_load.extend([lgb_load_mean, lgb_load_std])

if USE_XGBOOST:
    xgb_gen_mean, xgb_gen_std = create_meta_features({'xgb': xgb_gen_models}, X_train, 'xgb')
    xgb_load_mean, xgb_load_std = create_meta_features({'xgb': xgb_load_models}, X_train, 'xgb')
    meta_train_gen.extend([xgb_gen_mean, xgb_gen_std])
    meta_train_load.extend([xgb_load_mean, xgb_load_std])

if USE_CATBOOST:
    cb_gen_mean, cb_gen_std = create_meta_features({'cb': cb_gen_models}, X_train, 'cb')
    cb_load_mean, cb_load_std = create_meta_features({'cb': cb_load_models}, X_train, 'cb')
    meta_train_gen.extend([cb_gen_mean, cb_gen_std])
    meta_train_load.extend([cb_load_mean, cb_load_std])

# Stack meta features
X_meta_gen = np.column_stack(meta_train_gen)
X_meta_load = np.column_stack(meta_train_load)

print(f"Meta features shape: {X_meta_gen.shape}")

# Train meta-learners (simple linear model to avoid overfitting)
print("\nTraining meta-learners...")
meta_model_gen = Ridge(alpha=1.0)
meta_model_gen.fit(X_meta_gen, y_train_gen)

meta_model_load = Ridge(alpha=1.0)
meta_model_load.fit(X_meta_load, y_train_load)

print("✅ Meta-learners trained")

# PREDICTION AND SUBMISSION

In [ ]:
# ============================================
# 8. GENERATE TEST PREDICTIONS
# ============================================

print("\n" + "="*60)
print("GENERATING TEST PREDICTIONS")
print("="*60)

# Create meta features for test
meta_test_gen = []
meta_test_load = []

# LightGBM test predictions
lgb_test_gen = np.mean([m.predict(X_test, num_iteration=m.best_iteration)
                        for m in lgb_gen_models], axis=0)
lgb_test_load = np.mean([m.predict(X_test, num_iteration=m.best_iteration)
                         for m in lgb_load_models], axis=0)

# For stacking
lgb_test_gen_std = np.std([m.predict(X_test)
                           for m in lgb_gen_models], axis=0)
lgb_test_load_std = np.std([m.predict(X_test)
                            for m in lgb_load_models], axis=0)

meta_test_gen.extend([lgb_test_gen, lgb_test_gen_std])
meta_test_load.extend([lgb_test_load, lgb_test_load_std])

if USE_XGBOOST:
    xgb_test_gen = np.mean([m.predict(X_test) for m in xgb_gen_models], axis=0)
    xgb_test_load = np.mean([m.predict(X_test) for m in xgb_load_models], axis=0)
    xgb_test_gen_std = np.std([m.predict(X_test) for m in xgb_gen_models], axis=0)
    xgb_test_load_std = np.std([m.predict(X_test) for m in xgb_load_models], axis=0)
    meta_test_gen.extend([xgb_test_gen, xgb_test_gen_std])
    meta_test_load.extend([xgb_test_load, xgb_test_load_std])

if USE_CATBOOST:
    cb_test_gen = np.mean([m.predict(X_test) for m in cb_gen_models], axis=0)
    cb_test_load = np.mean([m.predict(X_test) for m in cb_load_models], axis=0)
    cb_test_gen_std = np.std([m.predict(X_test) for m in cb_gen_models], axis=0)
    cb_test_load_std = np.std([m.predict(X_test) for m in cb_load_models], axis=0)
    meta_test_gen.extend([cb_test_gen, cb_test_gen_std])
    meta_test_load.extend([cb_test_load, cb_test_load_std])

# Get stacked predictions
X_meta_test_gen = np.column_stack(meta_test_gen)
X_meta_test_load = np.column_stack(meta_test_load)

stacked_gen_pred = meta_model_gen.predict(X_meta_test_gen)
stacked_load_pred = meta_model_load.predict(X_meta_test_load)

# Also calculate simple average (often works well)
if USE_XGBOOST and USE_CATBOOST:
    avg_gen_pred = (lgb_test_gen + xgb_test_gen + cb_test_gen) / 3
    avg_load_pred = (lgb_test_load + xgb_test_load + cb_test_load) / 3
elif USE_XGBOOST:
    avg_gen_pred = (lgb_test_gen + xgb_test_gen) / 2
    avg_load_pred = (lgb_test_load + xgb_test_load) / 2
else:
    avg_gen_pred = lgb_test_gen
    avg_load_pred = lgb_test_load

# Choose best ensemble method (you can experiment)
USE_STACKING = True  # Set to False to use simple average

if USE_STACKING:
    final_gen_normalized = stacked_gen_pred
    final_load_normalized = stacked_load_pred
    print("Using stacked predictions")
else:
    final_gen_normalized = avg_gen_pred
    final_load_normalized = avg_load_pred
    print("Using averaged predictions")

# ============================================
# 9. DENORMALIZE AND CREATE SUBMISSION
# ============================================

print("\n" + "="*60)
print("CREATING SUBMISSION")
print("="*60)

# Denormalize predictions
final_gen_pred = final_gen_normalized * test_processed['panels_capacity_W'].values
final_load_pred = final_load_normalized * test_processed['load_capacity_W'].values

# Ensure non-negative
final_gen_pred = np.maximum(0, final_gen_pred)
final_load_pred = np.maximum(0, final_load_pred)

print(f"Generation range: {final_gen_pred.min():.2f} - {final_gen_pred.max():.2f} W")
print(f"Load range: {final_load_pred.min():.2f} - {final_load_pred.max():.2f} W")

# Create submission
submission = test_df_original[['test_id', 'generation_W', 'load_W']].copy()

# Replace -1 values
mask_gen_predict = (submission['generation_W'] == -1)
mask_load_predict = (submission['load_W'] == -1)

submission.loc[mask_gen_predict, 'generation_W'] = final_gen_pred[mask_gen_predict]
submission.loc[mask_load_predict, 'load_W'] = final_load_pred[mask_load_predict]

# Handle -2 values
if (sample_submission['generation_W'] < 0).sum() == 0:
    submission.loc[submission['generation_W'] == -2, 'generation_W'] = 0
    submission.loc[submission['load_W'] == -2, 'load_W'] = 0

# Ensure match with sample submission
final_submission = sample_submission[['test_id']].merge(
    submission, on='test_id', how='left'
)

for col in ['generation_W', 'load_W']:
    if col in sample_submission.columns:
        final_submission[col] = final_submission[col].fillna(sample_submission[col])
    final_submission[col] = final_submission[col].fillna(0.0).clip(lower=0)

# Save
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
submission_filename = f'submission_stacked_{timestamp}.csv'
final_submission.to_csv(submission_filename, index=False)
final_submission.to_csv('submission_stacked.csv', index=False)

print(f"✅ Submission saved as: {submission_filename}")

# CONLCUSION

In [ ]:
# ============================================
# 10. FEATURE IMPORTANCE ANALYSIS
# ============================================

print("\n" + "="*60)
print("FEATURE IMPORTANCE")
print("="*60)

# Get average feature importance from LightGBM models
importance_gen = np.mean([m.feature_importance(importance_type='gain')
                          for m in lgb_gen_models], axis=0)
importance_load = np.mean([m.feature_importance(importance_type='gain')
                           for m in lgb_load_models], axis=0)

# Create importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance_gen': importance_gen,
    'importance_load': importance_load,
    'importance_avg': (importance_gen + importance_load) / 2
}).sort_values('importance_avg', ascending=False)

print("\n📊 Top 15 Most Important Features:")
print(importance_df.head(15)[['feature', 'importance_avg']])

# Save importance for analysis
importance_df.to_csv('feature_importance.csv', index=False)

print("\n" + "="*60)
print("✅ ADVANCED STACKING PIPELINE COMPLETE!")
print("="*60)

print(f"""
📊 Models Used:
- LightGBM: {len(lgb_gen_models)} folds
{"- XGBoost: " + str(len(xgb_gen_models)) + " folds" if USE_XGBOOST else ""}
{"- CatBoost: " + str(len(cb_gen_models)) + " folds" if USE_CATBOOST else ""}
- Meta-learner: {"Stacking with Ridge" if USE_STACKING else "Simple Average"}
""")